In [1]:
import pandas as pd
import numpy as np
import torch
from torchvision.datasets import Caltech256
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers

from library.taxonomy_constructors import SyntheticTaxonomy, CrossPredictionsTaxonomy
from library.models import ResNetModel
from library.datasets import Caltech256MappedDataModule

# Load Caltech256 dataset information
caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
caltech256_targets = pd.read_csv("data/caltech256.csv")

# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

In [2]:
# Create synthetic taxonomy with 3 domains
synthetic_taxonomy = SyntheticTaxonomy.create_synthetic_taxonomy(
    num_atomic_concepts=257,
    num_domains=3,
    domain_class_count_mean=180,
    domain_class_count_variance=10,
    concept_cluster_size_mean=5,
    concept_cluster_size_variance=1,
    no_prediction_class=True,
    atomic_concept_labels=caltech256_labels,
    relationship_type="true",
)

# Extract domain mappings
domain_A_mapping = synthetic_taxonomy.domains[0].to_mapping()
domain_B_mapping = synthetic_taxonomy.domains[1].to_mapping()
domain_C_mapping = synthetic_taxonomy.domains[2].to_mapping()

print(f"Domain A classes: {len(set(domain_A_mapping.values()))}")
print(f"Domain B classes: {len(set(domain_B_mapping.values()))}")
print(f"Domain C classes: {len(set(domain_C_mapping.values()))}")

Domain A classes: 38
Domain B classes: 39
Domain C classes: 34


In [3]:
# Configuration
TRAIN = False  # Set to True to train models from scratch


def train_domain_model(domain_mapping, domain_name, logger_name, model_name):
    """Train a ResNet model for a specific domain"""
    tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name=logger_name)
    dataset = Caltech256MappedDataModule(mapping=domain_mapping)

    trainer = Trainer(
        max_epochs=50,
        logger=tb_logger if TRAIN else False,
        callbacks=[
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="val_loss",
                mode="min",
                save_top_k=1,
                filename=model_name,
                enable_version_counter=False,
            )
        ],
    )

    if TRAIN:
        model = ResNetModel(
            num_classes=len(set(domain_mapping.values())),
            architecture="resnet50",
            optim="sgd",
            optim_kwargs={
                "lr": 0.01,
                "momentum": 0.9,
                "weight_decay": 5e-4,
            },
        )
        trainer.fit(model, datamodule=dataset)
        results = trainer.test(datamodule=dataset, ckpt_path="best")
    else:
        model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
        results = trainer.test(model, datamodule=dataset)

    print(f"{domain_name} Results: {results}")
    return results

In [4]:
# Train all three domain models
print("Training/Testing Domain A Model:")
domain_A_results = train_domain_model(
    domain_A_mapping,
    "Domain A",
    "caltech256_synthetic_2_A",
    "resnet50-caltech256-synthetic-2-A-min-val-loss",
)

print("\nTraining/Testing Domain B Model:")
domain_B_results = train_domain_model(
    domain_B_mapping,
    "Domain B",
    "caltech256_synthetic_2_B",
    "resnet50-caltech256-synthetic-2-B-min-val-loss",
)

print("\nTraining/Testing Domain C Model:")
domain_C_results = train_domain_model(
    domain_C_mapping,
    "Domain C",
    "caltech256_synthetic_2_C",
    "resnet50-caltech256-synthetic-2-C-min-val-loss",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training/Testing Domain A Model:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 35/35 [00:09<00:00,  3.82it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Testing DataLoader 0: 100%|██████████| 35/35 [00:08<00:00,  4.09it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.8085493445396423
        eval_loss           0.8723654747009277
        hp_metric           0.8085493445396423
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Domain B Results: [{'eval_loss': 0.8723654747009277, 'eval_accuracy': 0.8085493445396423, 'hp_metric': 0.8085493445396423}]

Training/Testing Domain C Model:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 31/31 [00:07<00:00,  4.06it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.8045685291290283
        eval_loss           0.7620490193367004
        hp_metric           0.8045685291290283
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Domain C Results: [{'eval_loss': 0.7620490193367004, 'eval_accuracy': 0.8045685291290283, 'hp_metric': 0.8045685291290283}]


In [5]:
# Configuration for prediction generation
PREDICT = False  # Set to True to generate predictions from scratch

if PREDICT:
    # Load datasets for all three domains
    dataset_domain_A = Caltech256MappedDataModule(mapping=domain_A_mapping)
    dataset_domain_B = Caltech256MappedDataModule(mapping=domain_B_mapping)
    dataset_domain_C = Caltech256MappedDataModule(mapping=domain_C_mapping)

    # Load trained models for all three domains
    model_domain_A = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-caltech256-synthetic-2-A-min-val-loss.ckpt"
    )
    model_domain_A.eval()

    model_domain_B = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-caltech256-synthetic-2-B-min-val-loss.ckpt"
    )
    model_domain_B.eval()

    model_domain_C = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-caltech256-synthetic-2-C-min-val-loss.ckpt"
    )
    model_domain_C.eval()

    trainer = Trainer(logger=False, enable_checkpointing=False)

    print("Generating cross-domain predictions...")

    # Generate all cross-domain predictions (6 combinations)
    model_A_on_domain_B = trainer.predict(model_domain_A, datamodule=dataset_domain_B)
    model_A_on_domain_C = trainer.predict(model_domain_A, datamodule=dataset_domain_C)
    model_B_on_domain_A = trainer.predict(model_domain_B, datamodule=dataset_domain_A)
    model_B_on_domain_C = trainer.predict(model_domain_B, datamodule=dataset_domain_C)
    model_C_on_domain_A = trainer.predict(model_domain_C, datamodule=dataset_domain_A)
    model_C_on_domain_B = trainer.predict(model_domain_C, datamodule=dataset_domain_B)

    # Convert predictions to class indices
    predictions_A_on_B = torch.cat(model_A_on_domain_B).argmax(dim=1)  # type: ignore
    predictions_A_on_C = torch.cat(model_A_on_domain_C).argmax(dim=1)  # type: ignore
    predictions_B_on_A = torch.cat(model_B_on_domain_A).argmax(dim=1)  # type: ignore
    predictions_B_on_C = torch.cat(model_B_on_domain_C).argmax(dim=1)  # type: ignore
    predictions_C_on_A = torch.cat(model_C_on_domain_A).argmax(dim=1)  # type: ignore
    predictions_C_on_B = torch.cat(model_C_on_domain_B).argmax(dim=1)  # type: ignore

    # Get ground truth targets for all domains
    domain_A_targets = torch.cat(
        [label for _, label in dataset_domain_A.predict_dataloader()]
    )
    domain_B_targets = torch.cat(
        [label for _, label in dataset_domain_B.predict_dataloader()]
    )
    domain_C_targets = torch.cat(
        [label for _, label in dataset_domain_C.predict_dataloader()]
    )

    # Save all prediction results
    pd.DataFrame(
        {
            "domain_A": domain_A_targets,
            "predictions_B_on_A": predictions_B_on_A,
            "predictions_C_on_A": predictions_C_on_A,
        }
    ).to_csv("data/caltech256_3domain_A_predictions.csv", index=False)

    pd.DataFrame(
        {
            "domain_B": domain_B_targets,
            "predictions_A_on_B": predictions_A_on_B,
            "predictions_C_on_B": predictions_C_on_B,
        }
    ).to_csv("data/caltech256_3domain_B_predictions.csv", index=False)

    pd.DataFrame(
        {
            "domain_C": domain_C_targets,
            "predictions_A_on_C": predictions_A_on_C,
            "predictions_B_on_C": predictions_B_on_C,
        }
    ).to_csv("data/caltech256_3domain_C_predictions.csv", index=False)

    print("All predictions saved to CSV files.")

# Load the prediction results
df_A_3domain = pd.read_csv("data/caltech256_3domain_A_predictions.csv")
df_B_3domain = pd.read_csv("data/caltech256_3domain_B_predictions.csv")
df_C_3domain = pd.read_csv("data/caltech256_3domain_C_predictions.csv")

print(f"Domain A predictions shape: {df_A_3domain.shape}")
print(f"Domain B predictions shape: {df_B_3domain.shape}")
print(f"Domain C predictions shape: {df_C_3domain.shape}")

Domain A predictions shape: (2204, 3)
Domain B predictions shape: (2199, 3)
Domain C predictions shape: (1970, 3)


In [6]:
# Construct taxonomy from all cross-domain predictions (3 domains = 6 prediction pairs)
taxonomy_3domain = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=[
        # Domain A → Domain B
        (0, 1, np.array(df_B_3domain["predictions_A_on_B"], dtype=np.intp)),
        # Domain A → Domain C
        (0, 2, np.array(df_C_3domain["predictions_A_on_C"], dtype=np.intp)),
        # Domain B → Domain A
        (1, 0, np.array(df_A_3domain["predictions_B_on_A"], dtype=np.intp)),
        # Domain B → Domain C
        (1, 2, np.array(df_C_3domain["predictions_B_on_C"], dtype=np.intp)),
        # Domain C → Domain A
        (2, 0, np.array(df_A_3domain["predictions_C_on_A"], dtype=np.intp)),
        # Domain C → Domain B
        (2, 1, np.array(df_B_3domain["predictions_C_on_B"], dtype=np.intp)),
    ],
    domain_targets=[
        (0, np.array(df_A_3domain["domain_A"], dtype=np.intp)),
        (1, np.array(df_B_3domain["domain_B"], dtype=np.intp)),
        (2, np.array(df_C_3domain["domain_C"], dtype=np.intp)),
    ],
    domain_labels=synthetic_taxonomy.domain_labels,
    relationship_type="hypothesis",
)

print("3-domain taxonomy constructed from cross-domain predictions.")

3-domain taxonomy constructed from cross-domain predictions.


In [8]:
# Generate and save taxonomy visualizations
print("Generating 3-domain taxonomy visualizations...")

taxonomy_3domain.visualize_graph("3-Domain Caltech256 Model Taxonomy").save_graph(
    "output/caltech256_3domain_synthetic_model_taxonomy.html"
)

synthetic_taxonomy.visualize_graph(
    "3-Domain Caltech256 Ground Truth Taxonomy"
).save_graph("output/caltech256_3domain_synthetic_taxonomy.html")
fig = synthetic_taxonomy.visualize_3d_graph(
    show_labels=False,  # Labels make graph unreadable in 3D
).show()

print("3-domain taxonomy visualizations saved to output/ directory.")

Generating 3-domain taxonomy visualizations...


3-domain taxonomy visualizations saved to output/ directory.


In [ ]:
# Evaluate the 3-domain taxonomy against the ground truth
edr_3domain = taxonomy_3domain.edge_difference_ratio(synthetic_taxonomy)
precision_3domain, recall_3domain, f1_3domain = taxonomy_3domain.precision_recall_f1(
    synthetic_taxonomy
)

print("3-Domain Synthetic Taxonomy Evaluation:")
print(f"Edge Difference Ratio: {edr_3domain:.4f}")
print(f"Precision: {precision_3domain:.4f}")
print(f"Recall: {recall_3domain:.4f}")
print(f"F1 Score: {f1_3domain:.4f}")

In [ ]:
# Build and visualize universal taxonomies for the 3-domain case
print("Building 3-domain universal taxonomies...")

taxonomy_3domain.build_universal_taxonomy()
taxonomy_3domain.visualize_graph(
    "3-Domain Caltech256 Model Universal Taxonomy"
).save_graph("output/caltech256_3domain_synthetic_model_universal_taxonomy.html")

synthetic_taxonomy.build_universal_taxonomy()
synthetic_taxonomy.visualize_graph(
    "3-Domain Caltech256 Ground Truth Universal Taxonomy"
).save_graph("output/caltech256_3domain_synthetic_universal_taxonomy.html")

print("3-domain universal taxonomy visualizations saved to output/ directory.")